As usual, we'll start with cleaning and attempt to EDA at the end.

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('../data/FMCG_2022_2024.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '') #Clean the titles for ease of coding
print(df.shape)
df.head()

(190757, 14)


,date,sku,brand,segment,category,channel,region,pack_type,price_unit,promotion_flag,delivery_days,stock_available,delivered_qty,units_sold
0,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Retail,PL-Central,Multipack,2.38,0,1,141,128,9
1,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Retail,PL-North,Single,1.55,1,3,0,129,0
2,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Retail,PL-South,Carton,4.00,0,5,118,161,8
3,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Discount,PL-Central,Single,5.16,0,2,81,114,7
4,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Discount,PL-North,Single,7.66,0,4,148,204,12


All right, we have a formal 190kish sales records to look at. I'm unsure if the default order is appropriate to have sku first, but I guess it's akin to having people when dealing with some health dataset then getting more general from there. 14 initial features total. Depending upon what we do with this data, we'd likely set price as the target feature.

In [3]:
if sum(df.isnull().sum())==0:
    print("Phew, no nulls. Nothing else to say about that. Always a good policy to check though.")
else:
    print("Oh boy, we got some nulls to deal with. Let's get into the 'damage'.")
    print(df.columns[df.isnull().any()])

Phew, no nulls. Nothing else to say about that. Always a good policy to check though.


Honstly a shame in this case as I was hoping to also demo null handling. Oh well.

## Initial EDA (Exploratory Data Analysis)

In [4]:
i = -1

In [5]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(df[df.columns[i]].value_counts(normalize=False))

Now looking at feature 0, date.
~~~
date
2024-07-30    248
2024-06-16    247
2023-08-02    247
2023-08-31    246
2023-11-20    244
             ... 
2022-02-24     13
2022-01-24     13
2022-02-27     13
2022-01-22     12
2022-01-21      8
Name: count, Length: 1076, dtype: int64


Oh wow, they almost had a sale every single day in the three year period. So, they're obviously open every day of the week. Oh interesting, it seems that the first recorded data wasn't even until 2022-1-21. So, considering that February 2024 was a leap year then they had at last one sale every single day. Regardless, let's still figure out trends - weekday, month, year, and seasonal. Holidays might still affect sales too, but I doubt I'll get into that when this is just a demo.

In [6]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 1, sku.
~~~
sku has 30 categories.
sku
MI-006    0.043097
MI-026    0.043071
YO-029    0.041461
YO-005    0.041377
YO-012    0.040465
RE-004    0.040067
YO-014    0.039999
YO-001    0.039705
RE-007    0.039259
RE-015    0.038615
MI-023    0.037755
JU-021    0.036397
YO-009    0.035532
SN-027    0.033734
YO-003    0.033241
MI-022    0.032759
YO-016    0.031406
RE-025    0.031401
SN-010    0.030772
RE-017    0.030133
SN-013    0.028838
YO-020    0.028392
SN-019    0.027412
MI-002    0.026563
MI-011    0.025493
YO-024    0.025425
MI-008    0.025042
SN-028    0.024896
YO-018    0.024146
SN-030    0.023548
Name: proportion, dtype: float64


Retroactively, from the segment section:<br>
mi - milk<br>
yo - yogurt<br>
re - ready meal<br>
sn - snack bar<br>
ju - juice<br>



Somewhat evenly distributed where no one-product dominates. Ie even the smallest is not so much undersold than the top milk products. Also, we didn't even consider their price, rate of consumption, etc..

In [7]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 2, brand.
~~~
brand has 14 categories.
brand
SnBrand2    0.140362
YoBrand4    0.116992
YoBrand3    0.099504
MiBrand3    0.095556
YoBrand2    0.090948
ReBrand4    0.077874
YoBrand1    0.073706
MiBrand1    0.068590
ReBrand2    0.061534
MiBrand4    0.043071
ReBrand1    0.040067
JuBrand3    0.036397
SnBrand3    0.028838
MiBrand2    0.026563
Name: proportion, dtype: float64


I'm unsure if brands do multiple things - ie does milk brand 1 also sell yogurt (under brand 1)? Anyways, of interest later will be to formula add them together and compare stuff. Ie Sn, whatever that is, clearly win ovrall. Yet how much within their own sn department do they dominate?

In [8]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 3, segment.
~~~
segment has 13 categories.
segment
Yogurt-Seg1       0.140760
Yogurt-Seg3       0.130071
Milk-Seg2         0.126365
Yogurt-Seg2       0.110318
Milk-Seg3         0.080852
ReadyMeal-Seg1    0.077874
SnackBar-Seg1     0.075856
ReadyMeal-Seg2    0.070199
SnackBar-Seg2     0.059610
Juice-Seg3        0.036397
SnackBar-Seg3     0.033734
ReadyMeal-Seg3    0.031401
Milk-Seg1         0.026563
Name: proportion, dtype: float64


Hmm, what did this really change? Regardless, we at least see a formal clarification of each thing now.<br><br>
Yeah, I'm honestly unsure at the moment how these segments are working. Ie from a quick scan above (in the brand sectin) yogurts took up approximately 40% of sales. That is still true now, yet earlier it was with 4 brands and here with just 3 categories that don't appear to be sums of the more granular brand. So, brand, as an analogy Greek Yogurt, does not seem to be a complete sub-category of whatever segment is. Per a ChatGPT query it would seem to be which type of yogurt ie vanilla, chocolate, etc.. Hence, evrythingmaks sense.
<br> But maybe not - there only appears to be one type of juice.... and it's assigned to segment3. So, that seem to imply Seg3 across the 5 items (yogurt, milk, readymeal, snackbar, and juice) are similar in flavor/type. So too regarding Seg1 and 2, seemingly.
<br><br>
Let this be a great example of the utility of domain knowledge. However, I'd say I'm doing pretty welll looking at this type of data professionally for the first time ever.

In [9]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 4, category.
~~~
category has 5 categories.
category
Yogurt       0.381150
Milk         0.233779
ReadyMeal    0.179474
SnackBar     0.169200
Juice        0.036397
Name: proportion, dtype: float64


Unsurprising to me that yogurt is the most purchased item per buying habbits and then milk per that and preservability. A bit surpised that snack bars aren't higher and likely now I can accurately transition into an interesting observation about this dataset - we're in Pland. Oh, I think I already mentioned that. Regardless, it's true - likely their eating (as well as perhaps the raw status of yogurt for example - maybe it's just a lot better over there, let alone cultural differences that may exist and the like).

In [10]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 5, channel.
~~~
channel has 3 categories.
channel
Retail        0.333870
E-commerce    0.333508
Discount      0.332622
Name: proportion, dtype: float64


About as even as you can get. Potentially might drop for simplicity, but perhaps across all facets it's not even. Ie maybe all snacks are retail, reflecting the spurious nature of snacking's definition.

In [11]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 6, region.
~~~
region has 3 categories.
region
PL-North      0.333644
PL-South      0.333235
PL-Central    0.333120
Name: proportion, dtype: float64


I'm also surprised that this is even. Albeit synthetic, but shocking. Yet, once again, specific products or prices might be skewed.

In [12]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 7, pack_type.
~~~
pack_type has 3 categories.
pack_type
Carton       0.333781
Multipack    0.333146
Single       0.333073
Name: proportion, dtype: float64


See above. Particularly when it would come to product ie milk I'd expect to see less multi-pack. Also curious if price would be correspondingly adjusted. I guess either-way, whethr price is gross or per-unit the point would be the same to except proportionally higher rates for singles. Cartons likely slightly lwoer and the cheapest being multi-pack.

In [13]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 8, price_unit.
~~~
count    190757.000000
mean          5.251979
std           2.166705
min           1.500000
25%           3.380000
50%           5.250000
75%           7.130000
max           9.000000
Name: price_unit, dtype: float64


Oh, yeah I guess they did go into 'unit' specificity. Honestly, I don't think I like that as $24/24 /= $1/1 as it still requires that spending of $24 for that one type of item. What are people's budgets? What is the life now for all of them to be used by this one customer?

Later we'll look into average yogurt... and the various devisions. I think for my sake we'll move this to the end for aesthetics.

In [14]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 9, promotion_flag.
~~~
promotion_flag has 2 categories.
promotion_flag
0    0.8508
1    0.1492
Name: proportion, dtype: float64


Seemingly if there was an add for the item - and phew it's not even.

Likely the item is new (unsure if we can confirm that from the data), not as popular (motivate sales), popular (potentially - ride the gravy train), and/or expensive. I mean, ultimately they hope to make money, so presumably whatever the sku's comparative features are...

In [15]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 10, delivery_days.
~~~
delivery_days has 5 categories.
delivery_days
4    0.202278
5    0.200286
1    0.199883
3    0.199327
2    0.198226
Name: proportion, dtype: float64


To the store or to the consumer? Seemingly to the store as recall 1/3 of the sales were retail. I'm unaware of the trucking and preserving process in general (let alone Poland) yet would ventue to still assume milk then yogurt would likely be the most affected by this. Unsure if juice or snack bars are even refridgerated at all. I'd also aassume, which I may be wrong, that the meals are frozen and likely have accomodating features to maintain that; eh, they're called 'ready meals' so likely at most refridgerated.

In [16]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 11, stock_available.
~~~
count    190757.000000
mean        157.697652
std          52.736104
min         -12.000000
25%         124.000000
50%         155.000000
75%         192.000000
max         405.000000
Name: stock_available, dtype: float64


Was this at the store? Ie 405 yogurts (example) on any one given transaction opportunity? Yeah, really unsure about this one. I do not see ay other infomation about this on Kaggle.

Hmm, and we have -12 for the minimum. Will likely need to change that. Presumably it was 12; well, maybe not. Perhaps they were supposed to bring 12 more than they had. So too below for delivred quantity.

In [17]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 12, delivered_qty.
~~~
count    190757.000000
mean        179.333655
std          40.037475
min         -11.000000
25%         152.000000
50%         179.000000
75%         206.000000
max         366.000000
Name: delivered_qty, dtype: float64


Hmm, seemingly related as they have similar yet smaller numbers. Of interest is that the mean (and median) is much highr for delivered though, implying that (given there's an order) it's a pretty sizable amount of the stock.

In [18]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 13, units_sold.
~~~
count    190757.000000
mean         19.919709
std          11.770077
min         -25.000000
25%          12.000000
50%          18.000000
75%          25.000000
max         139.000000
Name: units_sold, dtype: float64


Hmm, I'm guessing this is for the entire day. Also, bear in mind the multi-pack discussion.... We might change our minds once we see the prices though.
<br><br>
I'm guessing any negatives are a sign of returns.

## Feature Engineering

Well, at least round I of it.

In [22]:
df['date'].describe

<bound method NDFrame.describe of 0         2022-01-21
1         2022-01-21
2         2022-01-21
3         2022-01-21
4         2022-01-21
             ...    
190752    2024-12-31
190753    2024-12-31
190754    2024-12-31
190755    2024-12-31
190756    2024-12-31
Name: date, Length: 190757, dtype: object>

In [23]:
#Keep data whole, but break into weekday, month, and of couse year.

#Not going to bother to analyze Polish holidays when this is a demo, but will at least take note of them.
#and obviously re-analyze/analyze when tehy'e actualy made

df['date'] = pd.to_datetime(df['date'])

df['weekday']   = df['date'].dt.day_name()
df['monthday']  = df['date'].dt.day
df['month']     = df['date'].dt.month_name() 
df['year']      = df['date'].dt.year

Now let's formally look at the date components:

In [25]:
df['weekday'].value_counts(normalize=True)

weekday
Monday       0.143764
Sunday       0.143224
Tuesday      0.143224
Saturday     0.142632
Thursday     0.142553
Wednesday    0.142485
Friday       0.142118
Name: proportion, dtype: float64

Unsurprising given the observations earlir the near even split.

In [29]:
print(df['monthday'].value_counts(normalize=True, ascending=False)[:3])

df['monthday'].value_counts(normalize=True, ascending=True)[:3]

monthday
28    0.033315
24    0.033304
27    0.033299
Name: proportion, dtype: float64


monthday
31    0.019323
30    0.031108
5     0.032041
Name: proportion, dtype: float64

Recall that we don't always have day 31 in the Gregorian calendar. Otherwise, essentially evenly distributed.

In [30]:
df['month'].value_counts(normalize=True)

month
December     0.099021
October      0.094445
November     0.093192
August       0.090728
July         0.089795
September    0.088877
May          0.085077
June         0.084878
April        0.076044
March        0.070996
January      0.064585
February     0.062362
Name: proportion, dtype: float64

Hmmm,  we are dfinitly seeing some trends here. Hence the time series function. What's shocking is that it'snot even seasonal but seeemingly calendar as the earlier months floundr (all 3, in fact) and the later months spike up. Perhaps holidays justify that trend. Regardless, it's shocking. Furthermore, the trend continues as the next fewest months of sales are the next three months in the calendar!
<br>
It's definitly incumbent upon us to explore months with any other features. Of note also is that it's not perfectly calendar. Ie dec oct nov vs. dec nov oct, etc..

In [31]:
df['year'].value_counts(normalize=True)

year
2024    0.440141
2023    0.406533
2022    0.153326
Name: proportion, dtype: float64

Hmm, what happened in 2022? Definitely warrants attention

### Getting the various features data-ready

Actually, I might skip this if I don't decide to actually model as it'll make it easier for data visualziation purposes, which is what I'm going for right now, to leave them as is.

On the to-do list would be:

Categorical to dummify: category, channel, region, pack_type, weekday, month

Hmm, when it comes to sku, brands, segments...yeah, we'd also need to dummify them as ultimately leaving them as numbers would also skew any models as there's no inherent link between one sku being ca

In [32]:
#Per my aesthetics:

df_cleaned =  df.drop(['price_unit'], axis = 1)
df_cleaned['price_unit'] = df['price_unit']
df_cleaned.head()

,date,sku,brand,segment,category,channel,region,pack_type,promotion_flag,delivery_days,stock_available,delivered_qty,units_sold,weekday,monthday,month,year,price_unit
0,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Retail,PL-Central,Multipack,0,1,141,128,9,Friday,21,January,2022,2.38
1,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Retail,PL-North,Single,1,3,0,129,0,Friday,21,January,2022,1.55
2,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Retail,PL-South,Carton,0,5,118,161,8,Friday,21,January,2022,4.00
3,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Discount,PL-Central,Single,0,2,81,114,7,Friday,21,January,2022,5.16
4,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Discount,PL-North,Single,0,4,148,204,12,Friday,21,January,2022,7.66


TO do later

Aalyze brands

eye out for:
years, monnths